In [227]:
import json
import pandas as pd
import requests

# Retrieve data from API (top 20 tokens by biggest holders)
wallet_address='0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be'
url = "https://web3api.io/api/v1/addresses/" + wallet_address + "/tokens?size=20"
headers = {'content-type': 'application/json', 'x-api-key': '<ENTER_YOUR_API_KEY_HERE>'}

response = requests.get(url, headers=headers, verify=True)
if (not response.ok): # response.status_code
    response.raise_for_status()

# Parse JSON data
jData = json.loads(response.content)

# Load & transform JSON data into Data Frame
df = pd.DataFrame(jData['payload']['records'])
df.head(5)

,address,amount,decimals,holder,isERC20,isERC721,isERC777,isERC884,isERC998,name,symbol
0,0x131a99859a8bfa3251d899f0675607766736ffae,5094263311975284645421934999836566192569296108...,18,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,True,True,True,True,True,��,�؛A
1,0xa997b1167874ba010fe05772055815ddf203d86d,4000000000000000000000000000,18,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,True,False,False,False,False,Thunder Token,TT
2,0xf2eab3a2034d3f6b63734d2e08262040e3ff7b48,3278375664000000000000000000,18,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,True,False,False,False,False,CANDY,CANDY
3,0x6cb9daff67f07a93da266e81757cd1436cb3656d,1900010011000000000000000000,18,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,False,False,False,False,False,Matic Network tokens,MATIC
4,0x901cca6d2b8ee210484dc6ef1f1db71121c041b2,1900000000000000000000000000,18,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,True,False,False,False,False,Matic Token,MATIC


In [228]:
# Download token balances for each top 20 holders
balances = pd.DataFrame(columns=['symbol', 'timestamp', 'wallet_address', 'value'])
frames = []

for index, row in df.iloc[1:].iterrows():
    print(row['address'], row['symbol'], row['decimals'], row['amount'])

    # Download historical holders
    url = "https://web3api.io/api/v1/tokens/" + row['address'] + "/holders/historical?holderAddresses=" + wallet_address + "&timeFrame=30d&timeFormat=iso"
    response = requests.get(url, headers=headers, verify=True)
    if (not response.ok): # response.content, response.status_code
        response.raise_for_status()

    # Parse JSON data
    jData2 = json.loads(response.content)
    df2 = pd.DataFrame(jData2['payload']['data'])
    df2['address'] = row['address']
    df2['symbol'] = row['symbol']
    df2.rename({wallet_address: 'value'}, axis=1, inplace=True)
    frames.append(df2)

balances = pd.concat(frames, sort=False)
balances.head(5)

0xa997b1167874ba010fe05772055815ddf203d86d TT 18 4000000000000000000000000000
0xf2eab3a2034d3f6b63734d2e08262040e3ff7b48 CANDY 18 3278375664000000000000000000
0x6cb9daff67f07a93da266e81757cd1436cb3656d MATIC 18 1900010011000000000000000000
0x901cca6d2b8ee210484dc6ef1f1db71121c041b2 MATIC 18 1900000000000000000000000000
0x2ff2b86c156583b1135c584fd940a1996fa4230b findtherabbit.me 18 1553026016000000000000000000
0x713a1c4876673a982a828753166c452ba9721994 findtherabbit.me 18 1553024698000000000000000000
0xf0ef703a04cf96212031b51d373e8469687f308e ESC 18 1000000000000000000000000000
0x5c679a0a79d495affe049c02483519d51e37f32b DMHCO 18 1000000000000000000000000000
0xdbdd4517a581f173236157deea4be58918d6a059 SML 18 898993724000000000000000000
0x95efd1fe6099f65a7ed524def487483221094947 CBM 18 865457936676940760000000000
0x3506424f91fd33084466f402d5d97f05f8e3b4af CHZ 18 833092632308763625892621516
0xe685e67c37807dc57592d638778f18ea87a4031a CELR 18 597014925000000000000000000
0x52cf51b8d2d03251076d

,value,timestamp,address,symbol
0,4.000000e+27,1553817600000,0xa997b1167874ba010fe05772055815ddf203d86d,TT
0,3.278376e+27,1516665600000,0xf2eab3a2034d3f6b63734d2e08262040e3ff7b48,CANDY
0,1.900010e+27,1556236800000,0x6cb9daff67f07a93da266e81757cd1436cb3656d,MATIC
0,1.900000e+27,1555804800000,0x901cca6d2b8ee210484dc6ef1f1db71121c041b2,MATIC
0,1.553026e+27,1553040000000,0x2ff2b86c156583b1135c584fd940a1996fa4230b,findtherabbit.me


In [229]:
import datetime
import matplotlib.pyplot as plt

# balances['timestamp'] = balances['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
# balances['value'] = balances['value'].astype('float64')

symbols = balances['symbol'].unique()
symbol_indexes = {}
timestamps = balances['timestamp'].unique()
# print(symbols)

i = 0
for s in symbols:
    symbol_indexes[s] = i
    i += 1
    
token_balances = pd.DataFrame(columns=np.append(symbols, 'timestamp'))

pivot = {}
for s in symbols:
    for t in timestamps:
        if not t in pivot:
            pivot[t] = {}
        if balances.loc[(balances.symbol==s) & (balances.timestamp==t)].value.empty == False:
            pivot[t][s] = balances.loc[(balances.symbol==s) & (balances.timestamp==t)].value.values[0]
#         else:
#             pivot[t][s] = None

rows = []
for t in pivot.keys():
    row = [None] * (1 + symbols.size)
    for s in pivot[t].keys():
        row[0] = t
        row[symbol_indexes[s]] = pivot[t][s]
        rows.append(row)

token_balances = pd.DataFrame(rows, columns=np.append(['timestamp'], symbols))
token_balances.head(5)

,timestamp,TT,CANDY,MATIC,findtherabbit.me,ESC,DMHCO,SML,CBM,CHZ,CELR,VRA,FET(Fetch.AI),EOST,COCOS,NPXS,BOLT
0,4.000000e+27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,1.516666e+12,3.278376e+27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,1.556237e+12,NaN,1.900010e+27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,1.555805e+12,NaN,1.900000e+27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,1.553040e+12,NaN,NaN,1.553026e+27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
